In [1]:
import pandas

titanic = pandas.read_csv("kaggle_titanic/train.csv")
test_data = pandas.read_csv("kaggle_titanic/test.csv")

In [2]:
# preprocessing

import re
from sklearn import preprocessing

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

def get_age_group(age):
    if age < 15:
        return 0
    if age < 50:
        return 1
    return 2

scaler = preprocessing.MinMaxScaler()

titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
titanic["Embarked"] = titanic["Embarked"].fillna("S")

# new features
titanic["FamilySize"] = titanic["SibSp"] + titanic["Parch"]
titanic["NameLength"] = titanic["Name"].apply(lambda x: len(x))
titanic["Title"] = titanic["Name"].apply(get_title)


columns_labels = preprocessing.LabelEncoder()
train_columns_to_encode = ['Sex', 'Embarked', 'Title']

for col in train_columns_to_encode:
    titanic[col] = columns_labels.fit_transform(titanic[col])
    
titanic["Age"] = scaler.fit_transform(titanic["Age"])
# titanic["Embarked"] = scaler.fit_transform(titanic["Embarked"].astype(float))
# titanic["Parch"] = scaler.fit_transform(titanic["Parch"].astype(float))
# titanic["Pclass"] = scaler.fit_transform(titanic["Pclass"].astype(float))
    

In [17]:
# features exctractin

from sklearn import cross_validation
from sklearn.feature_selection import SelectKBest, f_classif

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked"]

# selector = SelectKBest(f_classif, k=5)
# selector.fit(titanic[predictors], titanic["Survived"])


In [4]:
def class_to_vect(classification):
    cls_count = 2
    vect = []
    for cls in classification:
        v = [0] * cls_count
        v[cls] = 1
        vect.append(v)
    return np.array(vect)


## Construct neural network:

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from sklearn.cross_validation import KFold
from keras.layers.core import Dense, Dropout, Activation

# alg = LogisticRegression(random_state=1)
# alg = GradientBoostingClassifier()
nn = Sequential()
nn.add(Dense(len(predictors), output_dim=5, activation='tanh'))
nn.add(Dense(5, output_dim=5, activation='tanh'))
nn.add(Dropout(0.1))
nn.add(Dense(5, output_dim=2, activation='softmax'))
nn.compile(loss='binary_crossentropy', optimizer='Adam')



## Evaluation:

In [13]:
def perc_score(actual, prediction):
    error = 0
    N = len(predictions)
    for i in range(N):
        if predictions[i] == actual.iloc[i]:
            error += 1
    return float(error) / N

kf = KFold(titanic.shape[0], n_folds=10, random_state=1)
predictions = []
for train, test in kf:
    train_predictors = titanic[predictors].as_matrix()[train,:]
    target = class_to_vect(titanic.Survived.as_matrix())
    target_train = class_to_vect(titanic.Survived.iloc[train].as_matrix())
    nn.fit(train_predictors, target[train], nb_epoch=100, show_accuracy=True)
    test_predictions = nn.predict_classes(titanic[predictors].as_matrix()[test,:])
    predictions.append(test_predictions)
predictions = np.concatenate(predictions)

print(perc_score(titanic.Survived, predictions))

0.8047138047138047


In [22]:
# processing test data 

test_data["Age"] = scaler.fit_transform(test_data["Age"].fillna(titanic["Age"].median()))
test_data["Embarked"] = test_data["Embarked"].fillna("S")
test_data["Fare"] = test_data["Fare"].fillna(titanic["Fare"].median())

# new features
test_data["FamilySize"] = test_data["SibSp"] + test_data["Parch"]
test_data["NameLength"] = test_data["Name"].apply(lambda x: len(x))
test_data["Title"] = test_data["Name"].apply(get_title)



for col in train_columns_to_encode:
    test_data[col] = columns_labels.fit_transform(test_data[col])


In [23]:
# predict test data

# alg.fit(titanic[predictors], titanic["Survived"])
nn.fit(titanic[predictors].as_matrix(), class_to_vect(titanic.Survived.as_matrix()), nb_epoch=100, show_accuracy=True)
predictions = nn.predict_classes(test_data[predictors].as_matrix())

submission = pandas.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": predictions
    })
submission.to_csv("kaggle_titanic/submission.csv", index=False)

Epoch 0
891/891 [==============================] - 0s - loss: 0.4683 - acc: 0.7969     
Epoch 1
891/891 [==============================] - 0s - loss: 0.4592 - acc: 0.8025     
Epoch 2
891/891 [==============================] - 0s - loss: 0.4656 - acc: 0.8002     
Epoch 3
891/891 [==============================] - 0s - loss: 0.4644 - acc: 0.8013     
Epoch 4
891/891 [==============================] - 0s - loss: 0.4594 - acc: 0.8047     
Epoch 5
891/891 [==============================] - 0s - loss: 0.4655 - acc: 0.8025     
Epoch 6
891/891 [==============================] - 0s - loss: 0.4515 - acc: 0.7946     
Epoch 7
891/891 [==============================] - 0s - loss: 0.4585 - acc: 0.8036     
Epoch 8
891/891 [==============================] - 0s - loss: 0.4562 - acc: 0.8025     
Epoch 9
891/891 [==============================] - 0s - loss: 0.4666 - acc: 0.8025     
Epoch 10
891/891 [==============================] - 0s - loss: 0.4564 - acc: 0.8070     
Epoch 11
891/891 [=============